In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import h5py
import pandas as pd
import pickle
import random

#this sets the size of the plot to something useful
plt.rcParams["figure.figsize"] = (20,15) 

In [2]:
file1 = open("qcd_100000_pt_1000_1200.pkl", 'rb')
data1 = pickle.load(file1)
file2 = open("higgs_100000_pt_1000_1200.pkl", 'rb')
data2 = pickle.load(file2)

## Problem 1
The first problem is to look at your background distribution and determine where the 5σ sensitivity threshold is. [Hint, you've done this several times now, look back at the first few labs.]

In [17]:
bkgd = stats.poisson.rvs(100, 2000)
signal = bkgd

probability = stats.poisson.sf(signal, lam)
sigma = stats.norm.isf(probability)
print("The significance of a 2000 jet signal is", sigma, "sigma.")

target_sigma = 5
new_prob = stats.norm.sf(target_sigma)
target_signal = stats.poisson.isf(new_prob, lam)
print("Jets needed for 5 sigma discovery:", target_signal)

probability = stats.poisson.sf(target_signal, lam)
sigma = stats.norm.isf(probability)
print("Sigma corresponding to", target_signal, ":" , sigma)

The significance of a 2000 jet signal is 2.100461227980275 sigma.
Jets needed for 5 sigma discovery: 2228.0
Sigma corresponding to 2228.0 : 5.020141370651767


## Problem 2
Now inject a signal of known strength. You will want to make your signal moderately strong, say somewhere in the 8-30σ range. Inject this signal into your background data many times.

In [5]:
size = 10000
noise = randn(1,size)*0.1;
signal = zeros(1,size)
signal(rand(size(signal),[1,500])) = 2
obssignal = noise + signal

SyntaxError: can't assign to function call (<ipython-input-5-8634e7928f74>, line 4)

a) Histogram how bright the observed signal appears to be, and discuss it's shape. Say in words what this histogram is telling you.

In [10]:
bkgd = stats.poisson.rvs(100, 2000)
size = 10000
noise = np.random.randn(1,size)*0.1
#signal(rand(size(signal),[1,500])) = 2
#Generate signal with a mean of 2, filling an array of size (size) with random values 1-500
signal = stats.poisson.rvs(2, 500)
obssignal = noise + signal

#Start with background (model/data)
data = bkgd + obssignal

#Inject fake signals of varying strength
signaledges = np.linspace(0,20,40)
dataedges = np.linspace(-7,27,68)

#Measure observed signal
Psd, temp, temp2= np.histogram2d(data, obssignal, bins=[dataedges,signaledges], density=True)

#Histogram true signal vs observed signal
datacenters = (dataedges[:-1] + dataedges[1:]) / 2
signalcenters = (signaledges[:-1] + signaledges[1:]) / 2
plt.pcolormesh(datacenters,signalcenters,Psd.T)
plt.ylabel('True signal, $P(s|d)$', fontsize = 24)
plt.xlabel('Observed data, $P(d|s)$', fontsize = 24)

ValueError: too many values to unpack (expected 2)

b) Is your observed signal biased? (e.g. is the observed value equally likely to be stronger or weaker than the true injected signal?) Is it symmetric?

The observed signal is not biased, and is symmetric.

## Problem 3
Now make a suite of injected signals. You will want to have a range of injected signal strengths, starting at zero and extending well above 5σ (30σ or more). You will want to follow the work you did for Homework 5 very closely.

In [3]:
probability = stats.poisson.sf(1000, 300)
sigma = stats.norm.isf(probability)
print("Sigma corresponding to", 300, ":" , sigma)

Sigma corresponding to 300 : 31.774720781417827


a) Clearly state what you are simulating, and make a 2D histogram of injected signal vs. observed data

b) For the same injected signal power as in problem 2, show that you get the same answer.

c) Now reverse the problem, select an observed data value (pick something quite a bit stronger than 5σ) and create a 1D histogram of the true signal probability given the observed data. Describe the meaning of this histogram.

d) For your observed signal, what is the 1σ uncertainty on the true signal strength?

e) Discuss the answer to part d in some depth. Is it symmetric? Is it biased? Does this make sense?